In [ ]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import bloomfilter
import block
import ecc
import helper
import network
import script
import tx
import network

In [ ]:
reload(network)
reload(bloomfilter)

import time

from random import randint
from io import BytesIO

from block import Block
from bloomfilter import BloomFilter
from helper import int_to_little_endian, encode_varint, read_varint, decode_base58, p2pkh_script, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import NetworkEnvelope, SimpleNode, GetHeadersMessage
from tx import Tx, TxIn, TxOut
from ecc import PrivateKey

private_key = PrivateKey(secret=61740721216174072121)
print(private_key.point.address(testnet=True))

addr = 'muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne'
h160 = decode_base58(addr)
bf = BloomFilter(100, 20, 20)
bf.add(h160)

target_address = 'muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 1000  # fee in satoshis

node = SimpleNode('tbtc.programmingblockchain.com', testnet=True)

# complete the handshake
node.handshake()

# load the bloom filter with the filterload command
node.send(b'filterload', bf.filterload())

# ask for blocks since block X
starting_block = bytes.fromhex('000000001439bf71cf731722c650375d0f188e528f5982c24a68a4745a809fd9')
getheaders_message = GetHeadersMessage(starting_block=starting_block)
node.send(b'getheaders', getheaders_message.serialize())

# wait for headers
headers_envelope = node.wait_for_commands({b'headers'})

# loop through each header and ask for the merkleblock
stream = BytesIO(headers_envelope.payload)
num_headers = read_varint(stream)
payload = encode_varint(num_headers)
last_block = None
for _ in range(num_headers):
    b = Block.parse(stream)
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    num_txs = read_varint(stream)
    if num_txs != 0:
        raise RuntimeError('got more than 0 txs')
    if last_block is not None and b.prev_block != last_block:
        raise RuntimeErrer('chain broken')
    # ask for merkleblock
    payload += int_to_little_endian(3, 4)
    payload += b.hash()[::-1]
    # set the last block
    last_block = b.hash()
node.send(b'getdata', payload)

# look for merkleblock or tx commands and validate them
tx_ins = []
total = 0
while True:
    envelope = node.wait_for_commands({b'merkleblock', b'tx'})
    stream = BytesIO(envelope.payload)
    if envelope.command.startswith(b'merkleblock'):
        mb = MerkleBlock.parse(stream)
        if not mb.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        t = Tx.parse(stream)
        for i, tx_out in enumerate(t.tx_outs):
            if tx_out.hash160() == h160:
                tx_ins.append(TxIn(t.hash()[::-1], i, b'', 0xffffffff))
                total += tx_out.amount
    if len(tx_ins) > 1:
        break
tx_outs = [TxOut(total-fee, target_script)]
t = Tx(1, tx_ins, tx_outs, 0, testnet=True)
for i in range(len(tx_ins)):
    t.sign_input(i, private_key, SIGHASH_ALL)

# first, add this transaction to our bloom filter
# t.hash()
# send this signed transaction on the network
node.send(b'tx', t.serialize())
print(t.serialize().hex())
# now wait for a response
envelope = node.wait_for_commands({b'inv', b'reject'})
print(envelope.command)
print(envelope.payload.hex())